# Task 6 – Text Generation with an LSTM Language Model on LABR

This notebook trains a word-level language model on the LABR Arabic book reviews and uses it to generate new Arabic text. The model is trained to predict the next word given a sequence of previous words (n-gram prefixes). At inference time, a seed sentence is provided and the model repeatedly predicts the next word, producing synthetic Arabic review text.

A separate tokenizer and LSTM architecture are used for the generation task. The corpus size, vocabulary size, and model capacity are limited to keep memory usage manageable while still capturing basic patterns in the review language.


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers

import matplotlib.pyplot as plt
import re

print("Available devices:", tf.config.list_physical_devices())


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
data_path = "data/reviews.tsv"  # adjust if necessary

df = pd.read_csv(
    data_path,
    sep="\t",
    header=None,
    names=["rating", "review_id", "user_id", "book_id", "text"],
)

len(df), df.head()


(63257,
    rating  review_id   user_id   book_id  \
 0       4  338670838   7878381  13431841   
 1       4   39428407   1775679   3554772   
 2       4   32159373   1304410   3554772   
 3       1  442326656  11333112   3554772   
 4       5   46492258    580165   3554772   
 
                                                 text  
 0   "عزازيل الذي صنعناه ،الكامن في أنفسنا" يذكرني...  
 1   من أمتع ما قرأت من روايات بلا شك. وحول الشك ت...  
 2   رواية تتخذ من التاريخ ،جوًا لها اختار المؤلف ...  
 3   إني أقدّر هذه الرواية كثيرا، لسبب مختلف عن أس...  
 4   الكاهن الذي أطلق على نفسه اسم هيبا تيمنا بالع...  )

In [3]:
arabic_diacritics = re.compile(r"[\u0617-\u061A\u064B-\u0652\u0670\u0640]+")

def clean_text(text):
    text = re.sub(arabic_diacritics, "", str(text))
    text = text.strip()
    return text

sample_raw = df.loc[0, "text"]
sample_clean = clean_text(sample_raw)
sample_raw, sample_clean


(' "عزازيل الذي صنعناه ،الكامن في أنفسنا" يذكرني يوسف زيدان بــ بورخس في استخدامه لحيلته الفنية،وخداع القاريء بأن الرواية ترجمة لمخطوط قديم. الهوامش المخترعة و اختلاق وجود مترجـِم عاد بي إلى بورخس و هوامشه و كتَّابه الوهميين. هذه أولى قراءاتي ليوسف زيدان ،وهو عبقري في السرد ويخلقُ جوَّا ساحرا متفرداً يغرقك في المتعة. هُنا يتجلى الشكُّ الراقي الممزوج بانسانية هيبا الفاتنة ربما تم تناول فكرة الرواية قبلاً ،ولكن هنا تفرداً و عذوبة لا تُقارن بنصٍ آخر كنتُ أودُّ لو صيغت النهاية بطريقة مختلفة فقد جاءت باردة لا تتناسب مع رواية خُطَّت بهذا الشغف . ولذا لا أستطيع منح الرواية خمس نجوم ،وإن كانت تجربة قرائية متفردة وممتعة. ',
 '"عزازيل الذي صنعناه ،الكامن في أنفسنا" يذكرني يوسف زيدان ب بورخس في استخدامه لحيلته الفنية،وخداع القاريء بأن الرواية ترجمة لمخطوط قديم. الهوامش المخترعة و اختلاق وجود مترجم عاد بي إلى بورخس و هوامشه و كتابه الوهميين. هذه أولى قراءاتي ليوسف زيدان ،وهو عبقري في السرد ويخلق جوا ساحرا متفردا يغرقك في المتعة. هنا يتجلى الشك الراقي الممزوج بانسانية هيبا الفاتنة ربما تم تناول فكر

## Preprocessing and corpus selection

The LABR dataset consists of Arabic book reviews with ratings and metadata. For text generation, only the review text is used. A simple normalization step is applied to remove Arabic diacritics and elongation marks and to trim extra whitespace, while leaving the rest of the text unchanged.

To keep memory usage under control, the generation model is trained on a subset of the reviews and a limited vocabulary. A new tokenizer is fitted on the cleaned review texts, restricted to the most frequent words. N-gram sequences are then constructed from each review, where each sequence contains a prefix of words and the task is to predict the next word.


In [4]:
# Limit the number of reviews used for language modelling
max_texts_for_generation = 10000
corpus_texts = df["text"].astype(str).tolist()[:max_texts_for_generation]

corpus_clean = [clean_text(t) for t in corpus_texts]

# Limit vocabulary size to reduce model size and memory usage
max_vocab_gen = 15000
gen_tokenizer = Tokenizer(num_words=max_vocab_gen, oov_token="<OOV>")
gen_tokenizer.fit_on_texts(corpus_clean)

total_words = min(max_vocab_gen, len(gen_tokenizer.word_index) + 1)
print("Total words in generation vocabulary:", total_words)

# Build n-gram input sequences from each cleaned review, with max length per review
input_sequences = []
max_tokens_per_review = 40

for line in corpus_clean:
    token_list = gen_tokenizer.texts_to_sequences([line])[0]
    if not token_list:
        continue
    token_list = token_list[:max_tokens_per_review]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[: i + 1]
        input_sequences.append(n_gram_sequence)

print("Number of n-gram sequences:", len(input_sequences))

# Pad sequences so they all have the same length (pre-padding)
max_sequence_len = max(len(seq) for seq in input_sequences)
input_sequences = np.array(
    pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")
)

# Predictors and integer labels (last word is the label)
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1].astype("int32")

xs.shape, labels.shape, max_sequence_len


Total words in generation vocabulary: 15000
Number of n-gram sequences: 246785


((246785, 39), (246785,), 40)

## N-gram sequences and training labels

Each cleaned review is tokenized into a sequence of word indices and truncated to a maximum of 40 tokens for the language modelling task. For each review, all possible n-grams are generated: for a sequence of length *L*, the prefixes of length 2, 3, …, *L* are collected as training examples. This produces a large set of sequences where the last word acts as the target and the preceding words form the input.

All n-gram sequences are pre-padded to the same maximum length using `pad_sequences`, so that the model always receives inputs of fixed size. The predictors `xs` contain all tokens except the last one, while the label vector `labels` contains the corresponding next-word indices. The labels are kept as integers and used with the sparse categorical cross-entropy loss to avoid constructing very large one-hot matrices.


In [ ]:
# Improved model hyperparameters
embedding_dim_gen = 128  # Increased from 32 for richer word representations
lstm_units_gen = 256     # Increased from 64 for more model capacity
dropout_rate = 0.3       # Regularization to prevent overfitting

gen_model = models.Sequential([
    # Embedding layer with larger dimension for better word representations
    layers.Embedding(total_words, embedding_dim_gen, input_length=max_sequence_len - 1),
    
    # First LSTM layer (returns sequences for stacking)
    layers.LSTM(lstm_units_gen, return_sequences=True, dropout=dropout_rate, recurrent_dropout=dropout_rate),
    
    # Second LSTM layer for deeper feature extraction
    layers.LSTM(lstm_units_gen, dropout=dropout_rate, recurrent_dropout=dropout_rate),
    
    # Dense layers with batch normalization and dropout for regularization
    layers.BatchNormalization(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(dropout_rate),
    layers.Dense(total_words, activation="softmax"),
])

optimizer_gen = optimizers.Adam(learning_rate=1e-3)

gen_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=optimizer_gen,
    metrics=["accuracy"],
)

gen_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 32)            480000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 15000)             495000    
                                                                 
Total params: 1,001,912
Trainable params: 1,001,912
Non-trainable params: 0
_________________________________________________________________


## Improved LSTM language model architecture

The language model is implemented as a stacked word-level LSTM network with several improvements for better text generation:

1. **Larger Embedding Dimension (128)**: The embedding layer maps each token index to a dense vector of size 128 (increased from 32), allowing richer semantic representations of words.

2. **Stacked LSTM Layers (2 × 256 units)**: Two LSTM layers are stacked, each with 256 hidden units. The first layer returns sequences to feed into the second, enabling the model to learn hierarchical temporal patterns. This deeper architecture captures more complex dependencies in the text.

3. **Dropout Regularization (0.3)**: Both dropout and recurrent dropout with a rate of 0.3 are applied in each LSTM layer to prevent overfitting and improve generalization.

4. **Batch Normalization**: Added before the dense layers to stabilize training and allow faster convergence.

5. **Larger Dense Layer (256 units)**: The final hidden state passes through a dense layer with 256 ReLU units, followed by dropout and a softmax output layer over the vocabulary.

The network is trained with the Adam optimizer and sparse categorical cross-entropy loss, treating next-word prediction as a multi-class classification problem.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

num_epochs_gen = 30  # Increased epochs (early stopping will prevent overfitting)
batch_size_gen = 128

# Callbacks for better training
callbacks = [
    # Stop training when validation loss stops improving
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    # Reduce learning rate when validation loss plateaus
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=2,
        min_lr=1e-6,
        verbose=1
    ),
]

history_gen = gen_model.fit(
    xs,
    labels,
    epochs=num_epochs_gen,
    batch_size=batch_size_gen,
    validation_split=0.1,  # Use 10% of data for validation
    callbacks=callbacks,
    verbose=2,
)


Epoch 1/15
1929/1929 - 399s - loss: 7.2096 - accuracy: 0.1420 - 399s/epoch - 207ms/step
Epoch 2/15
1929/1929 - 386s - loss: 6.9145 - accuracy: 0.1467 - 386s/epoch - 200ms/step
Epoch 3/15


KeyboardInterrupt: 

In [ ]:
def plot_gen_metric(history, metric):
    """Plot training and validation metrics."""
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[metric], label=f"Training {metric}")
    if f"val_{metric}" in history.history:
        plt.plot(history.history[f"val_{metric}"], label=f"Validation {metric}")
    plt.xlabel("Epochs")
    plt.ylabel(metric.capitalize())
    plt.title(f"Training and Validation {metric.capitalize()}")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

plot_gen_metric(history_gen, "accuracy")
plot_gen_metric(history_gen, "loss")


## Training behaviour

The accuracy and loss curves now show both training and validation metrics, providing insight into the model's generalization ability. Key improvements in the training process include:

1. **Validation Split (10%)**: A portion of the data is held out to monitor overfitting. The gap between training and validation curves indicates the degree of overfitting.

2. **Early Stopping**: Training automatically stops when validation loss stops improving for 5 consecutive epochs, preventing overfitting and saving computation time.

3. **Learning Rate Reduction**: The learning rate is automatically reduced by half when validation loss plateaus, allowing finer convergence in later training stages.

A steady increase in training accuracy together with decreasing loss indicates that the model is learning useful associations between prefixes and next words. The validation curves help identify when the model starts to memorize rather than generalize.


In [ ]:
# Reverse mapping from index to word for generation
reverse_word_index_gen = {idx: word for word, idx in gen_tokenizer.word_index.items()}

def sample_with_temperature(predictions, temperature=1.0):
    """
    Sample from the probability distribution with temperature scaling.
    
    - temperature < 1.0: More deterministic (sharper distribution)
    - temperature = 1.0: Sample from the original distribution
    - temperature > 1.0: More random (flatter distribution)
    """
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions + 1e-10) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

def top_k_sampling(predictions, k=10):
    """
    Sample from the top-k most probable words.
    This prevents sampling very unlikely tokens while maintaining diversity.
    """
    predictions = np.asarray(predictions).astype("float64")
    # Get indices of top k predictions
    top_k_indices = np.argsort(predictions)[-k:]
    # Zero out all other probabilities
    top_k_probs = np.zeros_like(predictions)
    top_k_probs[top_k_indices] = predictions[top_k_indices]
    # Renormalize
    top_k_probs = top_k_probs / np.sum(top_k_probs)
    # Sample from the top-k distribution
    return np.random.choice(len(predictions), p=top_k_probs)

def generate_text(seed_text, next_words=20, temperature=0.8, top_k=None, greedy=False):
    """
    Generate text by repeatedly predicting the next word given the current sequence.
    
    Args:
        seed_text: Starting text for generation
        next_words: Number of words to generate
        temperature: Controls randomness (lower = more deterministic)
        top_k: If set, sample only from top-k most likely words
        greedy: If True, always pick the most likely word (argmax)
    """
    text = clean_text(seed_text)
    for _ in range(next_words):
        token_list = gen_tokenizer.texts_to_sequences([text])[0]
        if not token_list:
            break
        token_list = pad_sequences(
            [token_list],
            maxlen=max_sequence_len - 1,
            padding="pre",
        )
        predicted_probs = gen_model.predict(token_list, verbose=0)[0]
        
        # Choose sampling strategy
        if greedy:
            predicted_index = int(np.argmax(predicted_probs))
        elif top_k is not None:
            predicted_index = int(top_k_sampling(predicted_probs, k=top_k))
        else:
            predicted_index = int(sample_with_temperature(predicted_probs, temperature))
        
        if predicted_index == 0:
            break
        next_word = reverse_word_index_gen.get(predicted_index, "")
        if not next_word or next_word == "<OOV>":
            break
        text += " " + next_word
    return text


In [ ]:
seed_sentences = [
    "هذا الكتاب",
    "القصة كانت",
    "أعتقد أن الكاتب",
]

print("=" * 80)
print("GREEDY SAMPLING (deterministic)")
print("=" * 80)
for seed in seed_sentences:
    generated = generate_text(seed, next_words=25, greedy=True)
    print("Seed:", seed)
    print("Generated:", generated)
    print("-" * 80)

print("\n" + "=" * 80)
print("TEMPERATURE SAMPLING (temperature=0.7)")
print("=" * 80)
for seed in seed_sentences:
    generated = generate_text(seed, next_words=25, temperature=0.7)
    print("Seed:", seed)
    print("Generated:", generated)
    print("-" * 80)

print("\n" + "=" * 80)
print("TOP-K SAMPLING (k=10)")
print("=" * 80)
for seed in seed_sentences:
    generated = generate_text(seed, next_words=25, top_k=10)
    print("Seed:", seed)
    print("Generated:", generated)
    print("-" * 80)


## Analysis of generated Arabic text

The improved language model demonstrates several sampling strategies for text generation:

### Sampling Strategies

1. **Greedy Sampling**: Always selects the most probable next word. This produces the most deterministic output but may lead to repetitive or generic text.

2. **Temperature Sampling**: Adjusts the "sharpness" of the probability distribution:
   - Lower temperature (e.g., 0.5): More focused on likely words, producing coherent but less diverse text
   - Higher temperature (e.g., 1.2): More random selection, producing more creative but potentially less coherent text

3. **Top-K Sampling**: Restricts selection to the K most probable words, then samples from this subset. This prevents the model from selecting very unlikely words while maintaining some diversity.

### Observations

The stacked LSTM with improved architecture and training produces text that:
- Uses frequent expressions and collocations learned from Arabic book reviews
- Shows better contextual understanding due to the deeper architecture
- Generates more varied outputs with temperature/top-k sampling compared to greedy decoding

The model still has limitations inherent to word-level language models: occasional grammatical inconsistencies, topic drift in longer sequences, and sensitivity to the sampling parameters. However, the improvements in architecture (stacked LSTMs, larger embeddings, regularization) and training (validation-based early stopping, learning rate scheduling) help produce more natural-sounding Arabic text compared to the baseline model.
